# Data drift dashboard in jupyter notebook

In [ ]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from sklearn import ensemble

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import DataDriftTab, NumTargetDriftTab, RegressionPerformanceTab

## Bicycle Demand Data

In [ ]:
content = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [ ]:
raw_data.head()

## Regression Model

### Feature engineering

In [ ]:
raw_data['month'] = raw_data.index.map(lambda x : x.month)
raw_data['hour'] = raw_data.index.map(lambda x : x.hour)
raw_data['weekday'] = raw_data.index.map(lambda x : x.weekday() + 1)

In [ ]:
raw_data.head()

### Model training

In [ ]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hour', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [ ]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [ ]:
reference.head()

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [ ]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

In [ ]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [ ]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

### Model Perfomance

In [ ]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [ ]:
regression_perfomance_dashboard = Dashboard(tabs=[RegressionPerformanceTab()])
regression_perfomance_dashboard.calculate(reference, None, column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

## Week 1

In [ ]:
regression_perfomance_dashboard.calculate(reference, current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                            column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

In [ ]:
target_drift_dashboard = Dashboard(tabs=[NumTargetDriftTab()])
target_drift_dashboard.calculate(reference, current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
target_drift_dashboard.show()

## Week 2

In [ ]:
regression_perfomance_dashboard.calculate(reference, current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                            column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

In [ ]:
target_drift_dashboard.calculate(reference, current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
target_drift_dashboard.show()

## Week 3

In [ ]:
regression_perfomance_dashboard.calculate(reference, current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                            column_mapping=column_mapping)

In [ ]:
regression_perfomance_dashboard.show()

In [ ]:
target_drift_dashboard.calculate(reference, current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
target_drift_dashboard.show()

## Data Drift

In [ ]:
column_mapping = ColumnMapping()

column_mapping.numerical_features = numerical_features

In [ ]:
data_drift_dashboard = Dashboard(tabs=[DataDriftTab()])
data_drift_dashboard.calculate(reference, current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                   column_mapping=column_mapping)

In [ ]:
data_drift_dashboard.show()

## Bringing in DVC

With the data drift getting worse, this is the perfect time to use DVC to version this new data and start running experiments to re-train the model on this new data.

```bash
$ dvc add hour.csv
$ dvc exp run
```

Then save this experiment and push it to your GitHub/GitLab repo with:

```bash
$ git add .
$ git commit -m "ran first experiment with new data"
$ git push
```

From here, you should be able to check out your results in [DVC Studio](https://studio.iterative.ai/)